In [ ]:
!pip install tqdm
!pip install opendatasets

In [ ]:
import os
import zipfile
import opendatasets as od

# Replace 'dataset_name' with the name of the dataset you want to download
od.download('https://www.kaggle.com/datasets/vermaavi/food11')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
EPOCHS = 100
BATCH_SIZE = 64
LR = 0.01

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
from PIL import Image

# Custom dataset that inherits from 
class Food11Dataset(Dataset):
    def __init__(self, dir, transform=None):
        self.dir = dir
        self.transform = transform
        self.file_list = [file for file in os.listdir(dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = os.path.join(self.dir, self.file_list[idx])
        
        image = Image.open(path)

        if self.transform:
            image = self.transform(image)

        label = int(self.file_list[idx].split('_')[0])

        return image, label

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 8)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 8)

        self.fc1 = nn.Linear(16 * 58 * 58, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 11)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        #print(f"conv1: {x.shape}")
    
        x = self.pool(x)
        #print(f"pool: {x.shape}")

        x = self.conv2(x)
        x = F.relu(x)
        #print(f"conv2: {x.shape}")

        x = self.pool(x)
        #print(f"pool: {x.shape}")

        # -1 means that pytorch will find it automatically
        x = x.view(-1, 16 * 58 * 58)

        x = self.fc1(x)
        x = F.relu(x)
        #print(f"fc1: {x.shape}")

        x = self.fc2(x)
        x = F.relu(x)
        #print(f"fc2: {x.shape}")

        x = self.fc3(x)
        #print(f"fc3: {x.shape}")

        return x

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

training_data = Food11Dataset('food11/training', transform=transform)

validate_data = Food11Dataset('food11/validation', transform=transform)

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)

validate_dataloader = DataLoader(validate_data, batch_size=BATCH_SIZE, shuffle=True)

classes = ('Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles/Pasta', 
           'Rice', 'Seafood','Soup', 'Vegetable/Fruit')    

In [ ]:
from tqdm import tqdm
from torch import optim
from torch import nn
from torch.utils.data import DataLoader

def train(model: nn.Module, dataloader: DataLoader, optimizer: optim.Optimizer, criterion: nn.Module):
    model.train()

    accumulated_loss = 0.0

    for images, labels in tqdm(dataloader):

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accumulated_loss += loss.item()
    
    return accumulated_loss / len(dataloader)


def validate(model: nn.Module, dataloader: DataLoader, criterion: nn.Module):
    model.eval()

    accumulated_loss = 0.0

    for images, labels in tqdm(dataloader):

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)

        loss = criterion(outputs, labels)

        accumulated_loss += loss.item()
    
    return accumulated_loss / len(dataloader)

In [ ]:
import torch.optim as optim
from tqdm import tqdm

model = Network()
model.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion)
    validate_loss = validate(model, train_dataloader, criterion)

    print(f"Epoch: {epoch}, Train loss: {train_loss}, Validate loss: {validate_loss}")
    
    # Save loss in a txt file loss.txt
    f = open("loss.txt", "a")
    f.write(str(train_loss) + "\n")
    f.close()
    
    # Save the last model
    torch.save(model.state_dict(), 'last.pth')

print("Finished training")

torch.save(model.state_dict(), 'final.pth')